In [1]:
from transformers import logging
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import torch
import torch.nn as nn
from torch.nn.functional import normalize, log_softmax

In [2]:
logging.set_verbosity_error()
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
from LamaTRExData import LamaTRExData
from DistributionComparison.DistributionComparisonResult import DistributionComparisonResult 
from DistributionComparison.SentenceTypologyEmbeddingDistributionComparison import SentenceTypologyEmbeddingDistributionComparison 
from ModelHelpers.fill_mask_helpers import get_distribution_from_pipeline
from relation_templates.templater import dict_templater

In [4]:
USELESS_WORD = "Erftwerk"
UNKNOWN_TOKEN = "[UNK]"
MASK = "[MASK]"
OBJ_LABEL = "obj-label"
SUBJ_LABEL = "sub-label"
VOCABULARY_SIZE = 28996
words = [ USELESS_WORD, UNKNOWN_TOKEN] 

In [5]:
relations = ["P19", "P36", "P101", "P178"]
#relations = ["P19"]

In [6]:
model = pipeline('fill-mask', model="bert-base-cased", top_k=VOCABULARY_SIZE)

In [7]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [8]:
def kl_divergence(P,Q):
    return (torch.log(P/Q)*P).sum().item()

In [9]:
@torch.no_grad()
def metric(left_sentence: str, right_sentence: str):
    Q_dist = get_distribution_from_pipeline(model(left_sentence))
    P_dist = get_distribution_from_pipeline(model(right_sentence))
    return kl_divergence(Q_dist,P_dist)

In [10]:
Comparer = SentenceTypologyEmbeddingDistributionComparison(relations, dict_templater, words, metric, MASK)

In [11]:
nr_of_compared_triplets = 20

In [ ]:
Comparer.compare(TREx.data, nr_of_compared_triplets)

 50%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                                                       | 10/20 [07:49<07:54, 47.47s/it]

In [ ]:
Comparer.print_comparison("simple", "simple", UNKNOWN_TOKEN)

In [ ]:
Comparer.print_comparison("simple", "complex", USELESS_WORD)

In [ ]:
Comparer.print_comparison("simple", "compound", USELESS_WORD)

In [ ]:
Comparer.print_comparison("simple", "compound-complex", SUBJ_LABEL)